In [ ]:
# ruff: noqa: E402 (no import at top level) suppressed on this file as we need to inject the truststore before importing the other modules
from dotenv import load_dotenv
from langchain.schema import StrOutputParser
from truststore import inject_into_ssl

load_dotenv()
inject_into_ssl()

# End of special imports

from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler

In [ ]:
chat_model = ChatOpenAI(model="gpt-4.1", temperature=0)
langfuse_handler = CallbackHandler()
langfuse_handler.auth_check()
chain = chat_model | StrOutputParser()
while True:
    question = input("User: ")
    if question.lower() == "exit":
        break
    elif question.strip() == "":
        print("System: Please enter a valid question.")
        continue
    output = chain.invoke(
        question,
        config={"callbacks": [langfuse_handler]},
    )
    print(f"LLM: {output}")
    langfuse_handler.flush()